# Jurisdiction Entry QA/QC Analysis

## Goals: 
    1. Given entries from a jurisdiction, track and analyze the changes from any previous table version.
        a. Determine Metrics that can be used as a reflection of valid entries
        b. Generalize to compare any updated entry with any master table enty which shares same unique key
        
    2. Find a method to track all rows potentially affected in redshift database via primary key
    
    3. Implement analysis as an AWS Lambda Function for external use.
        a. Gain a better understanding of "Layers" to bootstrap libraries in a space efficient manner
        b. Implement call capabilities both from a Python Environment and through a REST API
        
    4. Store warning and update logs into appropriate socrata tables for future reference.


## Assumptions:
    1. The tables ingested by this pipeline have been pulled from Socrata (and thus that values for all features are consistent in datatype)

    2. Jurisdiction updates are presented to the function as a json file.
    
    3. Aforementioned presentation shares same schema as old table
        a. Full table with updated columns
        b. Updated columns themselves
                
    4. If a row was added in the update, it was assigned a unique primary key
    
    5. A row's recid cannot be changed (since it's used as the primary key between old and new dataframe)
    
    6. Columns of type "float" are continuous (not ordinal/nominal categories)
    
    7. The fifth percentile to the 95th percentile of master socrata dataframe numerical columns is sufficient metric for "normal" updates
    
    8. Novel string updates when compared to old column counterparts are suspicious enough to raise warnings
    

## Ouput:

    1. Update table log with schema:
        - recid, city_name, cols_updated, updated_from_NaN, changed_vals, out_of_range, warnings, editor, edit_timestamp, edit_type (C/U/D), main_version, old_vals, new_vals
        
    2. Warning log with schema:
        - city, editor, recid, warn

# Use Case:

In [18]:
from table_update_insights import *

In [19]:
df_old = load_data()

pulling data in 1 chunks of 4936 rows each
pulling chunk 0
took 3.3616 seconds


In [23]:
dfuje = pd.DataFrame(load_json('updates_only_ex.json'))
event = {'socrata_data_id': 'qdrp-c5ra',
         'jurisdiction_entries' : dfuje}

In [32]:
dfuje

,recid,city_name,zn_code,zn_description,zn_area_overlay,regional_lu_class,max_far,max_dua,building_height,units_per_lot,editor,edit_date,county_name,zn_code_color,minimum_lot_sqft,source
0,thiswasadded1-f595-4fa8-8da2-975dfae46dc4,Cloverdale,M-1,General Industrial,None,5.0,0.0,0.0,NaN,NaN,Joshua Croff,2019-10-20 17:00:00,Sonoma,#75187C,NaN,NaN
3756,62db00d9-c9ff-4764-8d84-59413fe5f0b6,Oakland,CN-2 - D-KP-3,Commercial - Neighborhood Center,Kaiser Permanente Oakland Medical Center Zone,3.0,16.0,0.0,NaN,NaN,Joshua Croff,2019-10-24 17:00:00,Alameda,#FF0000,NaN,25.0
4001,f063d0c3-d09f-47e7-9381-f71b954ca0fe,Fremont,OS(Q),Open Space,Quarry Overlay,12.0,26.0,16.0,30.0,1.0,Marc Cleveland,2019-10-24 17:00:00,Alameda,#006A00,43560.0,NaN
4060,c1a59692-ee49-4677-95d0-f64e580e5754,Oakland,D-KP-1,Special and Combined - Kaiser Permanente Oakla...,None,4.0,0.0,0.0,NaN,12.0,Joshua Croff,2019-10-24 17:00:00,Alameda,#990000,10000.0,123.0
4115,dc4e140a-ed57-42c5-b4d0-89444919e88f,Dublin,C-N,Neighborhood Commercial,None,123.0,NaN,12.0,35.0,NaN,Michael Cass,2019-10-24 17:00:00,Alameda,#FF6B6B,5000.0,NaN
4149,e63d2e0d-2baf-4d4d-a6f8-5eae9bdbc33c,Fremont,P-87-2,Planned District,None,123.0,0.0,5000.0,50.0,NaN,Joshua Croff,2019-10-24 17:00:00,Alameda,#556B2F,NaN,NaN
4500,338fdfa3-3d33-42ec-afbb-b1ca11f7a73a,San Leandro,IT,Testing In Place,None,123.0,1.0,5000.0,50.0,NaN,Avalon Schultz,2019-10-24 17:00:00,Alameda,#B566FF,5000.0,NaN


In [24]:
update_info = CheckUpdates(df_old, dfuje, primary_key='recid', parcels=False, compare_masters=False)

In [26]:
update_info.update_analysis()

In [27]:
pretty_print(update_info.warning_table)

,city,editor,recid,warn
0,Fremont,Joshua Croff,e63d2e0d-2baf-4d4d-a6f8-5eae9bdbc33c,regional_lu_class value is out of range (val = 123.0)
1,Fremont,Joshua Croff,e63d2e0d-2baf-4d4d-a6f8-5eae9bdbc33c,max_dua value is out of range (val = 5000.0)
2,Oakland,Joshua Croff,62db00d9-c9ff-4764-8d84-59413fe5f0b6,max_far value is out of range (val = 16.0)
3,Oakland,Joshua Croff,c1a59692-ee49-4677-95d0-f64e580e5754,units_per_lot value is out of range (val = 12.0)
4,San Leandro,Avalon Schultz,338fdfa3-3d33-42ec-afbb-b1ca11f7a73a,regional_lu_class value is out of range (val = 123.0)
5,San Leandro,Avalon Schultz,338fdfa3-3d33-42ec-afbb-b1ca11f7a73a,max_dua value is out of range (val = 5000.0)
6,San Leandro,Avalon Schultz,338fdfa3-3d33-42ec-afbb-b1ca11f7a73a,"""Testing In Place"" not in other row records for zn_description"


In [28]:
pretty_print(update_info.update_log)

,recid,city_name,cols_updated,updated_from_NaN,changed_vals,out_of_range,warnings,editor,edit_timestamp,edit_type (C/U/D),main_version,old_vals,new_vals
0,thiswasadded1-f595-4fa8-8da2-975dfae46dc4,Cloverdale,row created,NaN,NaN,0,[],Joshua Croff,2019-10-20 17:00:00,C,1,NaN,"[thiswasadded1-f595-4fa8-8da2-975dfae46dc4, Cloverdale, M-1, General Industrial, None, 5.0, 0.0, 0.0, nan, nan, Joshua Croff, 2019-10-20 17:00:00, Sonoma, #75187C, nan, nan]"
0,dc4e140a-ed57-42c5-b4d0-89444919e88f,Dublin,"[zn_area_overlay, regional_lu_class, max_dua, edit_date]",-1,5,0,[],Michael Cass,2019-10-29 00:00:00,U,1,"[dc4e140a-ed57-42c5-b4d0-89444919e88f, Dublin, C-N, Neighborhood Commercial, nan, 3.0, nan, nan, 35.0, nan, Michael Cass, 2019-10-29 00:00:00, Alameda, #FF6B6B, 5000.0, nan]","[dc4e140a-ed57-42c5-b4d0-89444919e88f, Dublin, C-N, Neighborhood Commercial, None, 123.0, nan, 12.0, 35.0, nan, Michael Cass, 2019-10-24 17:00:00, Alameda, #FF6B6B, 5000.0, nan]"
0,e63d2e0d-2baf-4d4d-a6f8-5eae9bdbc33c,Fremont,"[zn_area_overlay, regional_lu_class, max_dua, building_height, edit_date]",1,4,2,"[regional_lu_class value is out of range (val = 123.0), max_dua value is out of range (val = 5000.0)]",Joshua Croff,2019-10-24 17:00:00,U,1,"[e63d2e0d-2baf-4d4d-a6f8-5eae9bdbc33c, Fremont, P-87-2, Planned District, nan, 11.0, 0.0, 0.0, nan, nan, Joshua Croff, 2019-05-24 00:00:00, Alameda, #556B2F, nan, nan]","[e63d2e0d-2baf-4d4d-a6f8-5eae9bdbc33c, Fremont, P-87-2, Planned District, None, 123.0, 0.0, 5000.0, 50.0, nan, Joshua Croff, 2019-10-24 17:00:00, Alameda, #556B2F, nan, nan]"
0,f063d0c3-d09f-47e7-9381-f71b954ca0fe,Fremont,"[regional_lu_class, max_far, max_dua, edit_date]",0,4,0,[],Marc Cleveland,2019-10-31 00:00:00,U,1,"[f063d0c3-d09f-47e7-9381-f71b954ca0fe, Fremont, OS(Q), Open Space, Quarry Overlay, 7.0, 0.0, 1.0, 30.0, 1.0, Marc Cleveland, 2019-10-31 00:00:00, Alameda, #006A00, 43560.0, nan]","[f063d0c3-d09f-47e7-9381-f71b954ca0fe, Fremont, OS(Q), Open Space, Quarry Overlay, 12.0, 26.0, 16.0, 30.0, 1.0, Marc Cleveland, 2019-10-24 17:00:00, Alameda, #006A00, 43560.0, nan]"
0,62db00d9-c9ff-4764-8d84-59413fe5f0b6,Oakland,"[max_far, edit_date, source]",1,2,1,[max_far value is out of range (val = 16.0)],Joshua Croff,2019-10-24 17:00:00,U,1,"[62db00d9-c9ff-4764-8d84-59413fe5f0b6, Oakland, CN-2 - D-KP-3, Commercial - Neighborhood Center, Kaiser Permanente Oakland Medical Center Zone, 3.0, 0.0, 0.0, nan, nan, Joshua Croff, 2019-05-22 00:00:00, Alameda, #FF0000, nan, nan]","[62db00d9-c9ff-4764-8d84-59413fe5f0b6, Oakland, CN-2 - D-KP-3, Commercial - Neighborhood Center, Kaiser Permanente Oakland Medical Center Zone, 3.0, 16.0, 0.0, nan, nan, Joshua Croff, 2019-10-24 17:00:00, Alameda, #FF0000, nan, 25.0]"
0,c1a59692-ee49-4677-95d0-f64e580e5754,Oakland,"[zn_area_overlay, units_per_lot, edit_date, minimum_lot_sqft, source]",3,2,1,[units_per_lot value is out of range (val = 12.0)],Joshua Croff,2019-10-24 17:00:00,U,1,"[c1a59692-ee49-4677-95d0-f64e580e5754, Oakland, D-KP-1, Special and Combined - Kaiser Permanente Oakland Medical, nan, 4.0, 0.0, 0.0, nan, nan, Joshua Croff, 2019-05-22 00:00:00, Alameda, #990000, nan, nan]","[c1a59692-ee49-4677-95d0-f64e580e5754, Oakland, D-KP-1, Special and Combined - Kaiser Permanente Oakland Medical, None, 4.0, 0.0, 0.0, nan, 12.0, Joshua Croff, 2019-10-24 17:00:00, Alameda, #990000, 10000.0, 123.0]"
0,338fdfa3-3d33-42ec-afbb-b1ca11f7a73a,San Leandro,"[zn_description, zn_area_overlay, regional_lu_class, max_dua, building_height, edit_date]",0,6,2,"[regional_lu_class value is out of range (val = 123.0), max_dua value is out of range (val = 5000.0), ""Testing In Place"" not in other row records for zn_description]",Avalon Schultz,2019-10-24 17:00:00,U,1,"[338fdfa3-3d33-42ec-afbb-b1ca11f7a73a, San Leandro, IT, Industrial Transition, nan, 5.0, 1.0, 40.0, 35.0, nan, Avalon Schultz, 2019-10-24 00:00:00, Alameda, #B566FF, 5000.0, nan]","[338fdfa3-3d33-42ec-afbb-b1ca11f7a73a, San Leandro, IT, Testing In Place, None, 123.0, 1.0, 5000.0, 50

In [31]:
update_info.update_log.to_csv('Update_Log_UseCase.csv', index=False)
update_info.warning_table.to_csv('Warning_Log_UseCase.csv', index=False)

# Lambda Function Evocation

In [33]:
dfuje = load_json('updates_only_ex.json')
event = {'socrata_data_id': 'qdrp-c5ra',
         'jurisdiction_entries' : dfuje}

In [12]:
import boto3
payload = json.dumps(event)
lambda_client = boto3.client('lambda')
lambda_payload = event
lambda_client.invoke(FunctionName='Jurisdiction-Entry-Logging', 
                     InvocationType='RequestResponse',
                     Payload=payload)

{'ResponseMetadata': {'RequestId': 'e7b0978b-068f-4e41-a321-dea8270f69f0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 10 May 2021 19:12:44 GMT',
   'content-type': 'application/json',
   'content-length': '211',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e7b0978b-068f-4e41-a321-dea8270f69f0',
   'x-amz-function-error': 'Unhandled',
   'x-amzn-remapped-content-length': '0',
   'x-amz-executed-version': '$LATEST',
   'x-amzn-trace-id': 'root=1-6099859f-52910e5f45cdbb8c5b564e72;sampled=0'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'FunctionError': 'Unhandled',
 'ExecutedVersion': '$LATEST',
 'Payload': <botocore.response.StreamingBody at 0x14c5a7190>}

# Deprecated!!!

# Simulating Jurisdiction Entries

- Using a few simulated cases, jurisdiction entries for three dates will be saved to "Example_Logs"

## Update 1:

In [ ]:
df_old = load_data()
update1 = use_cases(df_old)
update1_info = CheckUpdates(df_old, update1, primary_key='recid', parcels=False)

In [ ]:
update1_info.row_comparison
update1_info.update_analysis()

In [ ]:
update1_info.update_log

In [ ]:
pretty_print(update1_info.warning_table)

In [ ]:
date = pd.Timestamp('2019-10-25')
current_date = date.strftime("%y-%m-%d_%H-%M")

warnings_file_name = 'warnings_' + current_date
update_log_file_name = 'update-log_' + current_date

update1_info.update_log.to_csv('Example_Logs/'+update_log_file_name+'.csv')
update1_info.warning_table.to_csv('Example_Logs/'+warnings_file_name+'.csv')

In [ ]:
df_old.to_csv('Example_Logs/Master_Table.csv')

In [ ]:
pd.Timestamp('2019-10-25')

## Update 2:

In [ ]:
update2 = use_cases_entry_2(df_old)
update2_info = CheckUpdates(df_old, update2, primary_key='recid', parcels=False)

In [ ]:
#update_info.row_comparison
update2_info.update_analysis()

date = pd.Timestamp('2019-11-14')
current_date = date.strftime("%y-%m-%d_%H-%M")

warnings_file_name = 'warnings_' + current_date
update_log_file_name = 'update-log_' + current_date

update2_info.update_log.to_csv('Example_Logs/'+update_log_file_name+'.csv')
update2_info.warning_table.to_csv('Example_Logs/'+warnings_file_name+'.csv')

## Update 3:

In [ ]:
update3 = use_cases_entry_3(df_old)
update3_info = CheckUpdates(df_old, update3, primary_key='recid', parcels=False)

In [ ]:
#update_info.row_comparison
update3_info.update_analysis()

date = pd.Timestamp('2020-01-01')
current_date = date.strftime("%y-%m-%d_%H-%M")

warnings_file_name = 'warnings_' + current_date
update_log_file_name = 'update-log_' + current_date

update3_info.update_log.to_csv('Example_Logs/'+update_log_file_name+'.csv')
update3_info.warning_table.to_csv('Example_Logs/'+warnings_file_name+'.csv')

# Pulling the Warnings/Logs From Master to a Certain Date

- Product all the updates from the "master" table to the cutoff date in tabular format

In [ ]:
!pwd

In [ ]:
import os
#Pull the file list from "Example_Logs"

def retrieve_logs(path: str, cutoff_date: str) -> pd.DataFrame:
    """
    Inputs:
    path -> path to the logs
    cutoff_date -> Up to what day? format: yy-mm-dd_hh-mm
    """
    files = os.listdir(path)

    files_txt = [i for i in files if i.endswith('.csv')]

    cutoff_date = datetime.strptime(cutoff_date, '%y-%m-%d_%H-%M')

    warning_log_df = pd.DataFrame()
    update_log_df = pd.DataFrame()

    for file in files_txt[1:]:
        try:
            date = file.partition("_")[2].split(".")[0]
            date = datetime.strptime(date, '%y-%m-%d_%H-%M')
            if cutoff_date > date:
                if 'update' in file:
#                     print("update_log:")
#                     print(file)
                    update_log_df = update_log_df.append(pd.read_csv(path+"/"+file, index_col='Unnamed: 0'))
                elif 'warning' in file:
#                     print("warning_log:")
#                     print(file)
                    warning_log_df = warning_log_df.append(pd.read_csv(path+"/"+file, index_col='Unnamed: 0'))
        except:
            continue

    update_log_df = update_log_df.sort_values(by='city_name')
    warning_log_df = warning_log_df.sort_values(by='editor')
    return update_log_df, warning_log_df

path = '/Users/okeefe/Box/USF Data Science Practicum/2020-21/Okeefe/Project_3_BASIS_Pipeline/Example_Logs'
cutoff_date = '19-12-30_10-15'

update_log, warning_log = retrieve_logs(path, cutoff_date)

In [ ]:
pretty_print(update_log)

In [ ]:
pretty_print(warning_log)

# Roll Back to a Specified Date


# Save log into S3

- Once analysis is run, the following cell will save the file by date and time to the s3 path of choice.

In [ ]:
import datetime

now = datetime.datetime.now()
current_date = now.strftime("%y-%m-%d_%H:%M")

warnings_file_name = 'warnings_' + current_date
update_log_file_name = 'update_log_' + current_date



# post_df_to_s3(update_info.warning_table, 'upload-logs-data-lake-mtc', warnings_file_name)
# post_df_to_s3(update_info.update_table, 'upload-logs-data-lake-mtc', update_log_file_name)

def all_changed_rows(date):
Function: Logs before (date/time)

1. Open s3 bucket with all logs
2. Parsing all log document titles and extracting dates
3. Concatenate all log documents that are before/up to the inputted date/time

returns: two pandas dataframes

1. concatenated update log 
2. concatenated warning log (by city)



update_logs, warning_logs = all_changed_rows(date)

def rollback_table(update_logs, master_socrata_table, save_df=False):
  
1. Identify changed rows via update_logs
2. Make a copy of the master_socrata_table
3. Put it into dataframe
4. change the rows in the master_socrata_dataframe

(optional)
5. If save_df, save the updated master_socrata_dataframe as csv (upload to socrata?)

return changed_master_socrata_dataframe